# Task detector v3

This notebook aims to clean and improve the loop for training and detecting different tasks.

Here the steps would be:
- Define a set of datasets, each one from a different "task" using simulated data with the parameters found on the previous version of this notebook.
- Create a loop to go through all the different sets of data, identify the task to which it belongs and either use the model already trained on the identified task to predict the velocity or keep training the hnet using a new condition id.


Requirements:
- Keep information about all datasets (to which task they belong to and how they have been generated)
- Manage the storage of the different models and data used for the loop.

### Imports

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import os
import sys
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import math

# Imports DL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import *
from copy import deepcopy
import torch.utils.data as data
from torch.utils.data import Dataset


from hypnettorch.hnets import HyperNetInterface
from hypnettorch.hnets import HMLP


import copy


# Imports from other modules and packages in the project
sys.path.append('../')

from src.helpers import *
from src.visualize import *
from src.trainer import *
from src.trainer_hnet import * 
from src.helpers_task_detector import *
from Models.models import *

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append("c:\\Users\\nerea\\OneDrive\\Documentos\\EPFL_MASTER\\PDM\\Project\\PyalData")
# to change for the actual path where PyalData has been cloned

In [3]:
from pyaldata import *

In [4]:
name = 'Chewie'
date = '1007'
fold = 0
target_variable = 'vel'

### Load the data and generate the simulated datasets

We work on fold 0 for now.

In [6]:
## Load pre-processed data
data_path = '../Data/Processed_Data/Tidy_'+name+'_'+date+'.pkl'

with open(data_path, 'rb') as file:
    tidy_df = pickle.load(file)
baseline_df = tidy_df.loc[tidy_df['epoch'] == 'BL']

In [7]:
size_data = int(baseline_df.shape[0]/2)

In [8]:
size_data

8500

From those matrices, we will use half the data for one dataset and the other for a new one. The idea is that the model is exposed to the other half dataset and recognises the task it has already trained before.

In [9]:
x_train, y_train, x_val, y_val,\
      x_test, y_test, info_train, info_val,\
          info_test, list_mins_base, \
            list_maxs_base= get_dataset(baseline_df.iloc[:size_data, :], 
                                            fold, 
                                            target_variable= target_variable, 
                                            no_outliers = False, 
                                            force_data = True, 
                                            std = False)

Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0


In [10]:
datasets = {}
datasets['Data_'+str(0)+'_1'] = x_train, y_train, x_val, y_val, x_test, y_test
datasets['Data_'+str(0)+'_2'] = x_train, y_train, x_val, y_val, x_test, y_test

In [11]:

for i in range(1,5):
    data_matrix = np.vstack(baseline_df['both_rates'])
    baseline_df_sim = baseline_df.copy()
    if i == 1:
        sim_data = remove_neurons(data_matrix, 30)
    elif i==2:
        sim_data = shuffle_neurons(data_matrix, 60)
    elif i == 3:
        sim_data = add_gain(data_matrix,50)
    elif i == 4:
        sim_data = add_gain(data_matrix,50)
        
    baseline_df_sim['both_rates'] = sim_data.tolist()
    new_data = baseline_df_sim

    x_train, y_train, x_val, y_val,\
        x_test, y_test, info_train, info_val,\
            info_test, list_mins_base, \
                list_maxs_base= get_dataset(new_data.iloc[:size_data, :], 
                                                fold, 
                                                target_variable= target_variable, 
                                                no_outliers = False, 
                                                force_data = True, 
                                                std = False)
    datasets['Data_'+str(i)+'_1'] = [x_train, y_train, x_val, y_val, x_test, y_test,]

    x_train, y_train, x_val, y_val,\
        x_test, y_test, info_train, info_val,\
            info_test, list_mins_base, \
                list_maxs_base= get_dataset(new_data.iloc[size_data:, :], 
                                                fold, 
                                                target_variable= target_variable, 
                                                no_outliers = False, 
                                                force_data = True, 
                                                std = False)
    datasets['Data_'+str(i)+'_2'] = [x_train, y_train, x_val, y_val, x_test, y_test,]

Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0
Train trials 54
Test trials  17
Val trials 14
We are testing the optimization method on fold  0


In [12]:
data_dir = "../Data"
path_to_save_data = os.path.join(data_dir, 'Processed_Data', 'Detector_Ddatasets_'+name+'_'+str(date)+'.pkl')

# Pickle the data and save it to file
with open(path_to_save_data, 'wb') as handle:
    pickle.dump(datasets, handle, protocol=4)

#### Shuffle the datasets to have a random order for your task detector.

In [13]:

# Assuming `datasets` is your dictionary
keys_list = list(datasets.keys())
random.shuffle(keys_list)

shuffled_sets = datasets #{key: datasets[key] for key in keys_list}

In [14]:
shuffled_sets.keys()

dict_keys(['Data_0_1', 'Data_0_2', 'Data_1_1', 'Data_1_2', 'Data_2_1', 'Data_2_2', 'Data_3_1', 'Data_3_2', 'Data_4_1', 'Data_4_2'])

#### Model and hyperparameters definition

In [15]:
# Specify that we want our tensors on the GPU and in float32
device = torch.device('cuda:0') #suposed to be cuda
#device = torch.device('cpu') 
dtype = torch.float32

# Set the seed for reproducibility
seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)  # If using CUDA

# Define hyperparameters

#Hyperparameters objective and regularization
alpha_reg = 1e-5
l1_ratio_reg = 0.5

lr = 0.00001
loss_function = huber_loss
delta = 8  # hyperparameter for huber loss

# Hyperparameters RNN class
hidden_units = 300
num_layers = 1
input_size = 49
dropout = 0.2

#Other training hyperparameters

lr_gamma= 1.37 #for scheduler
lr_step_size = 10 #for scheduler

seq_length_LSTM= 19
batch_size_train= 25
batch_size_val = 25

torch.manual_seed(42)

#### Template x_train and y_train to get the dimensions of the matrices

In [16]:
num_features = datasets['Data_0_1'][0].shape[2]
num_dim_output = datasets['Data_0_1'][1].shape[2]

In [17]:
####### Define task detector model
task_detector_model =  Causal_Simple_RNN(num_features=num_features, 
                hidden_units= hidden_units, 
                num_layers = num_layers, 
                out_dims = num_dim_output,
                dropout = dropout).to(device)

In [18]:
#### Defining the template, main and hnet models and initializing them
template_m = Causal_Simple_RNN(num_features=num_features, 
                    hidden_units= hidden_units, 
                    num_layers = num_layers, 
                    out_dims = num_dim_output, ).to(device)

param_shapes = [p.shape for p in list(template_m.parameters())]

num_conditions = 60 # we want more possible conditions than what we can reach
size_task_embedding = 8 #to check if the best one

hnet = HMLP(param_shapes, uncond_in_size=0,
             cond_in_size=size_task_embedding,
            layers=[13], 
            num_cond_embs=num_conditions).to(device)

for param in hnet.parameters():
    param.requires_grad = True

hnet.apply_hyperfan_init()

w_test = hnet(cond_id = 0)

LSTM_ = False

model = RNN_Main_Model(num_features= num_features, hnet_output = w_test,  hidden_size = hidden_units,
                            num_layers= num_layers,out_dims=num_dim_output,  
                            dropout= dropout,  LSTM_ = LSTM_).to(device)

for param in model.parameters():
    param.requires_grad = False

Created MLP Hypernet.
Hypernetwork with 1823425 weights and 130202 outputs (compression ratio: 14.00).
The network consists of 1822945 unconditional weights (1822945 internally maintained) and 480 conditional weights (480 internally maintained).


In [19]:
# Set up the optimizer with the specified learning rate
optimizer = torch.optim.Adam(hnet.internal_params, lr=lr)

# Set up a learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, 
                                step_size=lr_step_size, 
                                gamma=lr_gamma)

### From here all in the loop

In [20]:
thrs = 0.8
calc_reg = False

In [21]:
predicted_tasks = []

In [22]:
for s in shuffled_sets.keys():

    #### Load data
    x_train, y_train, x_val, y_val, x_test, y_test = shuffled_sets[s]

    #################################################
    ####### Define max_id and model path ############
    #################################################

    path_recog_models = '../Models/Models_Task_Recognition_Loop'
    # Check if the directory exists, if not, create it
    if not os.path.exists(path_recog_models):
        os.makedirs(path_recog_models)

    trained_detectors = np.sort(os.listdir(path_recog_models))

    r2_list = []
    
    for i,m in enumerate(trained_detectors):
        model_i = torch.load(os.path.join(path_recog_models, m)).to(device)
        model_i.eval()
        _, _, _, r2_i,_ = eval_model(x_train, 
                                    y_train, 
                                    x_val, 
                                    y_val,
                                    x_test, 
                                    y_test,
                                    model_i, 
                                    metric = 'r2')
        r2_list.append(r2_i)

    if not r2_list:
        max_id = 0
        task_id = 0
        predicted_tasks.append([s,task_id])

        print('Training on the first task!')
        print('Task_id for this task is ', task_id)
        #Define the task detector model
        task_detector_i =  Causal_Simple_RNN(num_features=num_features, 
                    hidden_units= hidden_units, 
                    num_layers = num_layers, 
                    out_dims = num_dim_output,
                    dropout = dropout).to(device)
        # Training the task detector model
        train_losses, val_losses = \
            train_model(task_detector_i, 
                        x_train, 
                        y_train, 
                        x_val, 
                        y_val,
                        lr=  0.00001,
                        lr_step_size=lr_step_size,
                        lr_gamma= lr_gamma,
                        sequence_length_LSTM=seq_length_LSTM,
                        batch_size_train = batch_size_train,
                        batch_size_val = batch_size_val,
                        num_epochs=1000, 
                        delta = 8,                 
                        regularizer= Regularizer_RNN, 
                        l1_ratio = l1_ratio_reg,
                        alpha = alpha_reg,     
                        early_stop = 5)
        # Evaluate model on first seen data
        y_hat, y_true,train_score, v_score, test_score = eval_model( x_train, y_train,
                                                                    x_val, y_val,
                                                                    x_test, y_test, 
                                                                    task_detector_i, 
                                                                    metric = 'r2')
        print('R2 for the task', task_id, ' is ', v_score)

        if v_score <thrs:
            print('ERROR, THE TASK COULD NOT BE LEARNED BY THE DETECTOR')
            break
        else:
            print('Task learned without issues.')
        # Save the trained model
        save_model(task_detector_i, task_id, "Models_Task_Recognition_Loop")
        print('Training now on the hnet')
        train_losses_, val_losses_, best_w_ =train_current_task(
                                                            model, 
                                                            hnet,
                                                            y_train, 
                                                            x_train, 
                                                            y_val,
                                                            x_val, 
                                                            optimizer,
                                                            scheduler,
                                                            calc_reg = calc_reg,
                                                            cond_id = int(task_id),
                                                            lr=0.001,
                                                            lr_step_size=5,
                                                            lr_gamma= lr_gamma, #0.9
                                                            sequence_length_LSTM = seq_length_LSTM, #15
                                                            batch_size_train = batch_size_train, #15
                                                            batch_size_val = batch_size_train, #15
                                                            num_epochs=1000, 
                                                            delta = 8,
                                                            beta = 1e-1,             
                                                            regularizer=reg_hnet,
                                                            l1_ratio = l1_ratio_reg, #0.5
                                                            alpha = alpha_reg,    
                                                            early_stop = 5,
                                                            chunks = False)
        W_best = hnet(cond_id = task_id)
        r2, _ = calc_explained_variance_mnet(x_val, y_val, W_best, model)
        print('R2 for the HNET on Task ', task_id, ' is ', r2)
        # Save the trained model
        save_model(hnet, task_id, "HNET_Task_Recog_Loop")

            
    else:
        max_id = len(trained_detectors) - 1
        max_r2 = max(r2_list)

        if max_r2 > thrs:

            # Show performance on the hnet
            print('This data comes from a known task. ')
            task_id = np.argmax(r2_list)
            predicted_tasks.append([s,task_id])
            print('Task_id for this task is ', task_id)
            W_i = hnet(cond_id = int(task_id))
            r2,_ = calc_explained_variance_mnet(x_val, y_val, W_i, model)
            print('R2 for the HNET on task', task_id, ' is ', r2)

        else:
            if task_id >0:
                calc_reg = True
            print('This data comes from a different task !')
            max_id += 1
            print('max id has changed to ', max_id)
            task_id = max_id
            predicted_tasks.append([s,task_id])
            print('Task_id for this task is ', task_id)
            task_detector_i =  Causal_Simple_RNN(num_features=num_features, 
                        hidden_units= hidden_units, 
                        num_layers = num_layers, 
                        out_dims = num_dim_output,
                        dropout = dropout).to(device)

            # Training the task detector model
            train_losses, val_losses = \
                train_model(task_detector_i, 
                            x_train, 
                            y_train, 
                            x_val, 
                            y_val,
                            lr=  0.00001,
                            lr_step_size=lr_step_size,
                            lr_gamma= lr_gamma,
                            sequence_length_LSTM=seq_length_LSTM,
                            batch_size_train = batch_size_train,
                            batch_size_val = batch_size_val,
                            num_epochs=1000, 
                            delta = 8,                 
                            regularizer= Regularizer_RNN, 
                            l1_ratio = l1_ratio_reg,
                            alpha = alpha_reg,     
                            early_stop = 5)
            # Evaluate model on first seen data
            y_hat, y_true,train_score, v_score, test_score = eval_model( x_train, y_train,
                                                                        x_val, y_val,
                                                                        x_test, y_test, 
                                                                        task_detector_i, 
                                                                        metric = 'r2')
            print('R2 for the task', task_id, ' is ', v_score)

            if v_score <thrs:
                print('ERROR, THE TASK COULD NOT BE LEARNED BY THE DETECTOR')
                break
            else:
                print('Task learned without issues.')

            # Save the trained model
            save_model(task_detector_i, task_id, "Models_Task_Recognition_Loop")
            print('Training now on the hnet')
            train_losses_, val_losses_, best_w_ =train_current_task(
                                                                model, 
                                                                hnet,
                                                                y_train, 
                                                                x_train, 
                                                                y_val,
                                                                x_val, 
                                                                optimizer,
                                                                scheduler,
                                                                calc_reg = calc_reg,
                                                                cond_id = int(task_id),
                                                                lr=0.001,
                                                                lr_step_size=5,
                                                                lr_gamma= lr_gamma, #0.9
                                                                sequence_length_LSTM = seq_length_LSTM, #15
                                                                batch_size_train = batch_size_train, #15
                                                                batch_size_val = batch_size_train, #15
                                                                num_epochs=1000, 
                                                                delta = 8,
                                                                beta = 1e-1,             
                                                                regularizer=reg_hnet,
                                                                l1_ratio = l1_ratio_reg, #0.5
                                                                alpha = alpha_reg,    
                                                                early_stop = 5,
                                                                chunks = False)
            W_best = hnet(cond_id = task_id)
            r2,_ = calc_explained_variance_mnet(x_val, y_val, W_best, model)
            print('R2 for the HNET on Task ', task_id, ' is ', r2)
            # Save the trained model
            save_model(hnet, task_id, "HNET_Task_Recog_Loop")
    
    print(predicted_tasks)

Training on the first task!
Task_id for this task is  0
Decrease LR
Decrease LR
56
Train R2: 0.96 
Val R2: 0.86 
Test R2: 0.86 
R2 for the task 0  is  0.8615813205999203
Task learned without issues.
Training now on the hnet
Decrease LR
Decrease LR
R2 for the HNET on Task  0  is  0.8127052783966064
[['Data_0_1', 0]]
Train R2: 0.96 
Val R2: 0.86 
Test R2: 0.86 
This data comes from a known task. 
Task_id for this task is  0
R2 for the HNET on task 0  is  0.8174643814563751
[['Data_0_1', 0], ['Data_0_2', 0]]
Train R2: 0.75 
Val R2: 0.68 
Test R2: 0.68 
This data comes from a different task !
max id has changed to  1
Task_id for this task is  1
Decrease LR
Decrease LR
59
Train R2: 0.96 
Val R2: 0.85 
Test R2: 0.85 
R2 for the task 1  is  0.8494051714701392
Task learned without issues.
Training now on the hnet
Decrease LR
Decrease LR
R2 for the HNET on Task  1  is  0.7684815526008606
[['Data_0_1', 0], ['Data_0_2', 0], ['Data_1_1', 1]]
Train R2: 0.68 
Val R2: 0.74 
Test R2: 0.68 
Train R2: 0

In [23]:
task_id

4

In [25]:
for cond in range(0, task_id+1):
    W_i = hnet(cond_id = cond)
    data = datasets['Data_'+str(cond)+'_1']
    r2,_ = calc_explained_variance_mnet(data[4], data[5], W_i, model)
    print('R2 for the HNET on Task ', cond, ' is ', r2)

R2 for the HNET on Task  0  is  0.8013109564781189
R2 for the HNET on Task  1  is  0.776359349489212
R2 for the HNET on Task  2  is  0.8031349182128906
R2 for the HNET on Task  3  is  0.7994925677776337
R2 for the HNET on Task  4  is  0.7980213165283203


In [26]:
for cond in range(0, task_id+1):
    W_i = hnet(cond_id = cond)
    data = datasets['Data_'+str(cond)+'_2']
    r2,_ = calc_explained_variance_mnet(data[4], data[5], W_i, model)
    print('R2 for the HNET on Task ', cond, ' is ', r2)

R2 for the HNET on Task  0  is  0.8016993403434753
R2 for the HNET on Task  1  is  0.7778732776641846
R2 for the HNET on Task  2  is  0.8064682483673096
R2 for the HNET on Task  3  is  0.8167592585086823
R2 for the HNET on Task  4  is  0.7851686179637909
